In [1]:
from utils import setup_env_path

setup_env_path()

%load_ext autoreload
%autoreload 2

Former working directory:  /Users/ambroisebertin/Desktop/prog/prog_abeilles/fil-rouge-pollinisateurs/notebooks
Current working directory:  /Users/ambroisebertin/Desktop/prog/prog_abeilles/fil-rouge-pollinisateurs


In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import missingno as msno

In [4]:
API_keys = [
    "2b10Me1HF0rfjoGWCseolNa8e",
    "2b10Xs7brWPuBdRTSeWx9V7HJu",
    "2b10umm8L2jTYjlWrJKPfshJ0O",
    "2b10sEg5pSsrtVT372XqlrzcLe",
    "2b10LDWWlXJwuS6FleaqfK6Rke",
    "2b10BoBZKV8WqZx9XCYulKvdu",
    "2b10U9ilY13D9JvF70m7WjK1kO",
    "2b101ARTX7UKqG1hqaeL9qav",
    "2b10Aun0VZ0L3WxrRmMRtx6rsO",
    "2b10PHd7xZZfWqzPghtBrYY6Me",
    "2b10uRbRzoHPIDpZaY3ID9o4e",
    "2b104GXJh341DO2S60qo3ABkI",
]

On a 500 requêtes PlantNet par clef API chaque jour.

## Préparation des données

### sauter cette partie si les fichiers csv existent déjà dans temporary_data/plantes

A l'origine cette partie a été utilisée pour générer les datasets qui sont maintenant présents dans data/temporary_data/plantes. Ainsi, à moins de devoir recommencer de zéro, il n'est plus nécessaire d'executer ce code (mais il peut aider à comprendre le contenu des deux datasets construits).

In [ ]:
# Import spipoll.csv as a pandas dataframe
spipoll = pd.read_csv("././data/spipoll.csv",low_memory=False)

# Extract relevant columns from the spipoll dataframe
plantes = spipoll[['collection_id', 'plante_sc', 'plante_fr',
       'plante_precision', 'plante_inconnue', 'plante_caractere',
       'photo_fleur', 'photo_plante', 'photo_feuille']]

# Shrinking the data by grouping by collection_id
# Keep only the first row for each unique value of collection_id
plantes = plantes.drop_duplicates(subset='collection_id', keep='first')

plantes.shape

In [ ]:
values_to_check = ["Je ne sais pas", "Plante inconnue"]

In [ ]:
plantes_sc_unlabelled = plantes.loc[
    (
        (plantes["plante_sc"].isna() | plantes["plante_sc"].isin(values_to_check))
    )
    | ((plantes["plante_inconnue"] == 1.0))
]

# creation du dataframe contenant les valeurs de plantes mais pas de plantes_sc_unlabelled
plantes_sc_labelled = plantes[~plantes.index.isin(plantes_sc_unlabelled.index)]

plantes_sc_labelled.shape

In [ ]:
# verification des dimensions des deux df par rapport à la dim de plantes
plantes_sc_labelled.shape[0] + plantes_sc_unlabelled.shape[0] == plantes.shape[0]

In [ ]:
plantes_precision_unlabelled = plantes_sc_unlabelled.loc[
    (
        (plantes["plante_precision"].isna() | plantes["plante_precision"].isin(values_to_check))
    )
]

# creation du dataframe contenant les valeurs de plantes_sc_unlabelled mais pas de plantes_precision_unlabelled
plantes_precision_labelled = plantes_sc_unlabelled[~plantes_sc_unlabelled.index.isin(plantes_precision_unlabelled.index)]

plantes_precision_labelled.shape

In [ ]:
# verification des dimensions des deux df par rapport à la dim de plantes_sc_unlabelled
plantes_precision_labelled.shape[0] + plantes_precision_unlabelled.shape[0] == plantes_sc_unlabelled.shape[0]

Compression de l'information étiquettée redondante :

In [ ]:
# group plantes_sc_labelled by unique values of plante_sc
plantes_sc_labelled = plantes_sc_labelled.drop_duplicates(subset='plante_sc', keep='first')

# display the dimension of the dataframe
plantes_sc_labelled.shape

In [ ]:
# group plantes_precision_labelled by unique values of plante_precision
plantes_precision_labelled = plantes_precision_labelled.drop_duplicates(subset='plante_precision', keep='first')

# display the dimension of the dataframe
plantes_precision_labelled.shape

In [ ]:
# ajoute trois colonnes après plante_sc dans le dataframe plantes_sc_labelled "Famille", "Genre" et "Espèce" entre la colonne plante_sc et plante_fr
plantes_sc_labelled.insert(2, "Famille", "")
plantes_sc_labelled.insert(3, "Genre", "")
plantes_sc_labelled.insert(4, "Espece", "")

# ajoute trois colonnes après plante_sc dans le dataframe plantes_precision_labelled "Famille", "Genre" et "Espèce" entre la colonne plante_sc et plante_fr
plantes_precision_labelled.insert(2, "Famille", "")
plantes_precision_labelled.insert(3, "Genre", "")
plantes_precision_labelled.insert(4, "Espece", "")

# ajoute trois colonnes après plante_sc dans le dataframe plantes_precision_unlabelled "Famille", "Genre" et "Espèce" entre la colonne plante_sc et plante_fr
plantes_precision_unlabelled.insert(2, "Famille", "")
plantes_precision_unlabelled.insert(3, "Genre", "")
plantes_precision_unlabelled.insert(4, "Espece", "")

## Appel à l'API

In [ ]:
from data_quality.plant_treatment.plantnet_api import PlantNetPredictor

def call_API(df, iloc, api_key_index=0):
    
    predictor = PlantNetPredictor(key = API_keys[api_key_index])
    prediction = None

    try:
        prediction = predictor.predict(imageURL= df['photo_fleur'].iloc[iloc],
                        organs = "auto",
                        includeRelatedImages=False)
    except Exception as e:
        if api_key_index + 1 < len(API_keys):
            return call_API(df, iloc, api_key_index + 1)
        else:
            print(f"Error: {e}")
            return ["erreur_API", "erreur_API", "erreur_API"]

    try:
        famille = prediction["results"][0]["species"]["family"]["scientificNameWithoutAuthor"] # famille
        genre = prediction["results"][0]["species"]["genus"]["scientificNameWithoutAuthor"] # genre
        espece = prediction["results"][0]["species"]["scientificNameWithoutAuthor"] # espece
    
    except IndexError:
        print("IndexError: list index out of range")
        return ["erreur_API", "erreur_API", "erreur_API"]

    return [famille, genre, espece]

In [ ]:
def remplir_tableau(df, num):
    
    for i in tqdm(range(num)):
        start_index = 0

        for j, val in enumerate(df["Famille"]):
            if val == "":
                start_index = j
                break
        else:
            print("Le dataframe est déjà rempli.")
            return df

        try:
            df.iloc[start_index, 2:5] = call_API(df,start_index)
        except Exception as e:
            print(f"Error: {e}")

    return df

## Pipeline

Il faut environ 3 secondes pour un call API, soit environ 2h10 pour les 2620 plantes à traiter dans plantes_sc_labelled.

On va donc stocker le dataframe dans un csv entre chaque appel à l'API pour ne pas perdre le travail effectué.

In [ ]:
def save_dataframe(df, df_name):
    df.to_csv(f"data/temporary_data/plantes/{df_name}.csv", index=False)

In [5]:
def import_dataframe(df_name):
    return pd.read_csv(f"data/temporary_data/plantes/{df_name}.csv")

In [6]:
plantes_sc_labelled = import_dataframe("plantes_sc_labelled")
plantes_precision_labelled = import_dataframe("plantes_precision_labelled")
plantes_precision_unlabelled = import_dataframe("plantes_precision_unlabelled")

In [ ]:
def pipeline_remplissage(df_name, num):
    df = import_dataframe(df_name)
    df[["Famille", "Genre", "Espece"]] = df[["Famille", "Genre", "Espece"]].fillna("")
    remplir_tableau(df, num)
    save_dataframe(df, df_name)

In [ ]:
# POC
# pipeline_remplissage("plantes_precision_unlabelled", 10)

### Sauvegarde progressive des calls API

In [ ]:
def pipeline(df_name, n):
    for i in range(n):
        try:
            pipeline_remplissage(df_name, 10) # une itération prend 30 secondes
        except Exception as e:
            print(f"Error occurred after {i} successful runs: {e}")
            break   

permet d'effectuer une sauvegarde des telechargements toutes les 10 occurences ie toutes les 30 secondes

In [ ]:
pipeline('plantes_precision_unlabelled', 1)

### Verification de la progression des téléchargements

In [ ]:
def verification_progression(df_name):
    # importe le dataframe plantes_sc_labelled depuis le fichier csv
    df = pd.read_csv(f"data/temporary_data/plantes/{df_name}.csv")
    # utilise missingno pour visualiser les valeurs manquantes dans le dataframe
    msno.matrix(df)
    
    # estimation de la quantité de travail restante en téléchargement de données
    num_nan = df["Famille"].isna().sum()
    percent_nan = round((num_nan / len(df)) * 100,2)
    print("\n")
    print(f"Nombre de lignes dans 'Famille' restants à télécharger : {num_nan}")
    print(f"Pourcentage de lignes dans 'Famille' restants à télécharger : {percent_nan} %")

In [ ]:
# importe plantes_precision_unlabelled depuis le fichier csv
plantes_precision_unlabelled = import_dataframe("plantes_precision_unlabelled")
# toutes les lignes de famille genre espece conteant "erreur_API" doivent être vidées avec ""
plantes_precision_unlabelled[["Famille", "Genre", "Espece"]] = plantes_precision_unlabelled[["Famille", "Genre", "Espece"]].replace("erreur_API", "")
# sauvegarde le dataprecisioname dans un fichier csv
save_dataframe(plantes_precision_unlabelled, "plantes_precision_unlabelled")


In [ ]:
verification_progression('plantes_precision_unlabelled')

## Export des résultats

In [ ]:
def export_plantes(nom_df):
    # Importer le dataframe
    df = pd.read_csv(f"data/temporary_data/plantes/{nom_df}.csv")

    # Vérifier si la colonne "Famille" contient des valeurs NaN
    if df["Famille"].isna().any():
        num_nan = df["Famille"].isna().sum()
        percent_nan = round((num_nan / len(df)) * 100,2)
        print("Il reste", percent_nan,"% des données à télécharger, soit", num_nan,"lignes.")
        msno.matrix(df)
    else:
        # Sauvegarder le dataframe dans un fichier CSV
        df.to_csv(f"data/plantes/{nom_df}.csv", index=False)

In [ ]:
export_plantes('plantes_precision_unlabelled')

## Estimation de la qualité des réponses de PlantNet API

In [24]:
# compare les lignes des colonnes 'plante_sc'(en ne gardant que les strings de maximum trois mots inclus) et 'Espece' de plantes_sc_labelled
lignes_differentes_sc = plantes_sc_labelled[plantes_sc_labelled['plante_sc'] != plantes_sc_labelled['Espece']]

# ne garde que les colonnes 'collection_id', 'plante_sc' et 'Espece' dans lignes_differentes_sc
lignes_differentes_sc = lignes_differentes_sc[['collection_id', 'plante_sc', 'Espece']]

# supprime les lignes de lignes_differentes_sc où 'plante_sc' contient plus de deux fois ' ' dans la string
lignes_differentes_sc = lignes_differentes_sc[lignes_differentes_sc['plante_sc'].str.count(' ') == 1]

# sépare la colonne 'plante_sc' avec le séparateur ' ' et crée une nouvelle colonne 'plante_sc_bis' qui ne contient que le 2e mot
lignes_differentes_sc['plante_sc_bis'] = lignes_differentes_sc['plante_sc'].str.split(' ').str[1]
# sépare la colonne 'Espece' avec le séparateur ' ' et crée une nouvelle colonne 'espece_bis'
lignes_differentes_sc['espece_bis'] = lignes_differentes_sc['Espece'].str.split(' ').str[1]

# supprime les lignes de lignes_differentes_sc où plante_sc_bis est égal à espece_bis
lignes_differentes_sc = lignes_differentes_sc[lignes_differentes_sc['plante_sc_bis'] != lignes_differentes_sc['espece_bis']]

lignes_differentes_sc


,collection_id,plante_sc,Espece,plante_sc_bis,espece_bis
1,2,Acanthus mollis,Acanthus spinosus,mollis,spinosus
9,17,Senecio jacobaea,Senecio inaequidens,jacobaea,inaequidens
18,60,Euphorbia lathyris,Euphorbia characias,lathyris,characias
21,103,Ranunculus ficaria,Ficaria verna,ficaria,verna
28,206,Bergenia crassifolia,Bergenia cordifolia,crassifolia,cordifolia
...,...,...,...,...,...
2615,75 468,Arbutus andrachne,Arbutus unedo,andrachne,unedo
2616,75 525,Pavonia hastata,Hibiscus trionum,hastata,trionum
2617,75 554,Medicago falcata,Medicago sativa,falcata,sativa
2618,75 625,Helianthus divaricatus,Helianthus tuberosus,divaricatus,tuberosus


In [25]:
lignes_differentes_sc.shape[0] / plantes_sc_labelled.shape[0]

0.5015267175572519

In [26]:
# compare les lignes des colonnes 'plante_precision'(en ne gardant que les strings de maximum trois mots inclus) et 'Espece' de plantes_precision_labelled
lignes_differentes_precision = plantes_precision_labelled[plantes_precision_labelled['plante_precision'] != plantes_precision_labelled['Espece']]

# ne garde que les colonnes 'collection_id', 'plante_precision' et 'Espece' dans lignes_differentes_precision
lignes_differentes_precision = lignes_differentes_precision[['collection_id', 'plante_precision', 'Espece']]

# supprime les lignes de lignes_differentes_precision où 'plante_precision' contient plus de deux fois ' ' dans la string
lignes_differentes_precision = lignes_differentes_precision[lignes_differentes_precision['plante_precision'].str.count(' ') == 1]

# sépare la colonne 'plante_precision' avec le séparateur ' ' et crée une nouvelle colonne 'plante_precision_bis' qui ne contient que le 2e mot
lignes_differentes_precision['plante_precision_bis'] = lignes_differentes_precision['plante_precision'].str.split(' ').str[1]
# sépare la colonne 'Espece' avec le séparateur ' ' et crée une nouvelle colonne 'espece_bis'
lignes_differentes_precision['espece_bis'] = lignes_differentes_precision['Espece'].str.split(' ').str[1]

# supprime les lignes de lignes_differentes_precision où plante_precision_bis est égal à espece_bis
lignes_differentes_precision = lignes_differentes_precision[lignes_differentes_precision['plante_precision_bis'] != lignes_differentes_precision['espece_bis']]

lignes_differentes_precision

,collection_id,plante_precision,Espece,plante_precision_bis,espece_bis
4,66,Alysson maritime,Lobularia maritima,maritime,maritima
10,87,Chimonanthus praecox,Chimonanthus fragrans,praecox,fragrans
11,89,jasmin d'hiver,Jasminum nudiflorum,d'hiver,nudiflorum
15,98,Jasmin d'hiver,Jasminum nudiflorum,d'hiver,nudiflorum
20,124,Mahonia Charity,Berberis japonica,Charity,japonica
...,...,...,...,...,...
4293,75 168,Olearia paniculata,Buxus sempervirens,paniculata,sempervirens
4294,75 169,Callistemon rugulosus,Callistemon citrinus,rugulosus,citrinus
4296,75 390,Orpin reprise,Hylotelephium telephium,reprise,telephium
4307,75 791,Rosa polyantha,Rosa x odorata,polyantha,x


In [27]:
lignes_differentes_precision.shape[0] / plantes_precision_labelled.shape[0]

0.2672693555864627

## Décompression des données

On a à l'origine un dataset spipoll (670 744 lignes), groupé par collection_id (75 372 lignes) scindé en la partition plantes_sc_labelled et plantes_sc_unlabelled (respectivement 63 465 et 11 907 lignes). plantes_sc_unlabelled est scindé en la partition plantes_precision_labelled et plantes_precision_unlabelled (respectivement 9619 et 2288 lignes).


On avait compressé plantes_sc_labelled et plantes_precision_labelled en les groupant par valeur de plantes_sc et plantes_precision identiques (respectivement passé de 63 465 lignes à 2620 ; et passé de 9619 lignes à 4319). On va ici faire l'opération inverse pour compléter les lignes d'origine avec les valeurs obtenues par le traitement API.

In [ ]:
# on definit les deux dataframes à reconstruire
plantes_sc_labelled_long = plantes_sc_labelled # issus des dataframes vierges du début du notebook avant compression ...
plantes_precision_labelled_long = plantes_precision_labelled # ... pas des dataframes complétés par l'API !!

print(plantes_sc_labelled_long.shape[0], plantes_precision_labelled_long.shape[0])

In [ ]:
# ajout des colonnes "Famille", "Genre" et "Espece" dans les dataframes longs
plantes_sc_labelled_long.insert(2, "Famille", "")
plantes_sc_labelled_long.insert(3, "Genre", "")
plantes_sc_labelled_long.insert(4, "Espece", "")

plantes_precision_labelled_long.insert(2, "Famille", "")
plantes_precision_labelled_long.insert(3, "Genre", "")
plantes_precision_labelled_long.insert(4, "Espece", "")

In [ ]:
# on trie les dataframes longs par plantes_sc et plantes_precision
plantes_sc_labelled_long = plantes_sc_labelled_long.sort_values(by="plante_sc")
plantes_precision_labelled_long = plantes_precision_labelled_long.sort_values(by="plante_precision")

In [ ]:
plantes_sc_labelled_long.head()

In [ ]:
print(plantes_sc_labelled_long.shape[0], plantes_precision_labelled_long.shape[0])

In [ ]:
# on importe les petits dataframes qui ont été complétés avec l'API précédemment
plantes_sc_labelled = pd.read_csv(f"data/plantes/plantes_sc_labelled.csv")
plantes_precision_labelled = pd.read_csv(f"data/plantes/plantes_precision_labelled.csv")

print(plantes_sc_labelled.shape[0], plantes_precision_labelled.shape[0])

### Petit POC pour construire la fonction d'intérêt

In [ ]:
# prend les 50 premières lignes du dataframe plantes_sc_labelled_long et mets les dans un df POC
plantes_sc_labelled_long_POC = plantes_sc_labelled_long.head(50)
plantes_sc_labelled_long_POC.shape

In [ ]:
# Get the values of "Famille", "Genre", and "Espece" from the row in `plantes_sc_labelled` where the index is 856
famille, genre, espece = plantes_sc_labelled.loc[856, ["Famille", "Genre", "Espece"]]

# Identify the rows in `plantes_sc_labelled_long_POC` where "plante_sc" equals `plantes_sc_labelled["plante_sc"].iloc[856]`
mask = plantes_sc_labelled_long_POC["plante_sc"] == plantes_sc_labelled["plante_sc"].iloc[856]

# Replace the values of "Famille", "Genre", and "Espece" in these rows with the values from `plantes_sc_labelled`
plantes_sc_labelled_long_POC.loc[mask, "Famille"] = famille
plantes_sc_labelled_long_POC.loc[mask, "Genre"] = genre
plantes_sc_labelled_long_POC.loc[mask, "Espece"] = espece

In [ ]:
plantes_sc_labelled_long_POC.head(15)

### Construction de la fonction de décompression

In [ ]:
def remplacer_valeurs(df_court, df_long, colonne_interet):
    # Boucle sur chaque ligne dans `df_court`
    for i in tqdm(range(len(df_court))):
        # Obtient les valeurs de "Famille", "Genre", et "Espece" de la ligne dans `df_court` où l'index est i
        famille, genre, espece = df_court.loc[i, ["Famille", "Genre", "Espece"]]

        # Identifie les lignes dans `df_long` où "plante_sc" est égal à `df_court["plante_sc"].iloc[i]`
        masque = df_long[f"{colonne_interet}"] == df_court[f"{colonne_interet}"].iloc[i]

        # Remplace les valeurs de "Famille", "Genre", et "Espece" dans ces lignes avec les valeurs de `df_court`
        df_long.loc[masque, "Famille"] = famille
        df_long.loc[masque, "Genre"] = genre
        df_long.loc[masque, "Espece"] = espece

In [ ]:
# Appelle la fonction avec `plantes_sc_labelled` comme `df_court` et `plantes_sc_labelled_long_POC` comme `df_long`
remplacer_valeurs(plantes_sc_labelled, plantes_sc_labelled_long_POC, "plante_sc")

In [ ]:
remplacer_valeurs(plantes_sc_labelled, plantes_sc_labelled_long, "plante_sc")

In [ ]:
remplacer_valeurs(plantes_precision_labelled, plantes_precision_labelled_long, "plante_precision")

In [ ]:
# trie plantes_sc_labelled_long et plantes_precision_labelled_long par index croissant
plantes_sc_labelled_long = plantes_sc_labelled_long.sort_index()
plantes_precision_labelled_long = plantes_precision_labelled_long.sort_index()

In [ ]:
plantes_sc_labelled_long.shape

In [ ]:
plantes_precision_labelled_long.shape

In [ ]:
plantes_precision_unlabelled = pd.read_csv(f"data/plantes/plantes_precision_unlabelled.csv")
plantes_precision_unlabelled.shape

In [ ]:
# merge les dataframes plantes_sc_labelled_long, plantes_precision_labelled_long et plantes_precision_unlabelled
plantes_merged = pd.concat([plantes_sc_labelled_long, plantes_precision_labelled_long, plantes_precision_unlabelled])

# trie le dataframe plantes_merged par collection_id croissants
plantes_merged = plantes_merged.sort_values(by="collection_id")

# sauvegarde le dataframe plantes_merged dans le dossier temporary_data/plantes
plantes_merged.to_csv("data/temporary_data/plantes/plantes_merged.csv", index=False)

plantes_merged.shape

In [ ]:
plantes_merged.head(100)

## On va maintenant décompresser par collection_id

In [ ]:
# Import spipoll.csv as a pandas dataframe
spipoll = pd.read_csv("././data/spipoll.csv",low_memory=False)

# Extract relevant columns from the spipoll dataframe
plantes = spipoll[['collection_id', 'plante_sc', 'plante_fr',
       'plante_precision', 'plante_inconnue', 'plante_caractere',
       'photo_fleur', 'photo_plante', 'photo_feuille']]

# ajout des colonnes "Famille", "Genre" et "Espece" dans le dataframe plantes
plantes.insert(2, "Famille", "")
plantes.insert(3, "Genre", "")
plantes.insert(4, "Espece", "")

In [ ]:
plantes.shape

In [ ]:
# import de plantes_merged depuis le fichier csv
plantes_merged = pd.read_csv("data/temporary_data/plantes/plantes_merged.csv")
plantes_merged.shape

In [ ]:
remplacer_valeurs(plantes_merged, plantes, "collection_id")

In [ ]:
plantes.head()

In [ ]:
# save plantes in a csv file in data/plantes
plantes.to_csv("data/plantes/plantes.csv", index=False)